# Account Details

In [1]:
from utils.packages import *
warnings.filterwarnings("ignore")

## Read Yaml files

In [63]:
temp_file = "config/access_keys.yaml"
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)     

In [64]:
data = {}
data['instrument'] = "EUR_USD"
data["access_token"] = config["oanda_demo_hedge"]["token"]
data["accountID"] = config["oanda_demo_hedge"]["account_id"]
data["api"] = API(access_token = data["access_token"])

In [65]:
def get_ema_diff(granularity, ema_long, ema_short):    
    
    data["candle_param"] = {"count": ema_long, "granularity": granularity}
    data["candle_r"] = instruments.InstrumentsCandles(instrument=data['instrument'], params=data["candle_param"])
    data["api"].request(data["candle_r"])
    data["candle_data"] = data["candle_r"].response
    
    ema_l = pd.DataFrame([x['mid']['c'] for x in data["candle_data"]['candles']]).ewm(span=ema_long).mean()[0][ema_long - 1]
    ema_s = pd.DataFrame([x['mid']['c'] for x in data["candle_data"]['candles']]).ewm(span=ema_short).mean()[0][ema_short - 1]    
    return(abs(ema_s - ema_l))

In [66]:
%%time
get_ema_diff(granularity = 'M5', ema_long = 50, ema_short = 5)

Wall time: 563 ms


0.0014681604663813186

In [4]:
def get_avg_candle_height(candle_count, granularity):
    global data
    data["candle_param"] = {"count": candle_count, "granularity": granularity}
    data["candle_r"] = instruments.InstrumentsCandles(instrument=data['instrument'], params=data["candle_param"])
    data["api"].request(data["candle_r"])
    data["candle_data"] = data["candle_r"].response

    candle_height_list = []

    for candle in data["candle_data"]['candles']:
        high = np.float(candle['mid']['h'])
        low = np.float(candle['mid']['l'])
        candle_height_list.append(high - low) 

    data['avg_candle_height'] = np.round(np.mean(candle_height_list),5)
    return()

In [5]:
%%time
get_avg_candle_height(candle_count = 5, granularity = 'D')
print(f'avg_candle_height : {data["avg_candle_height"]}')

avg_candle_height : 0.00644
Wall time: 1.54 s


In [30]:
data['curr_date'] = None

In [35]:
today_date = datetime.today().date()
if data['curr_date'] != today_date:
    data['curr_date'] = today_date
    get_avg_candle_height(candle_count = 5, granularity = 'D')

In [ ]:
data = {}
data["balance_invest_ratio"] = 100
data["leverage"] = 50

data["access_token"] = config["oanda_demo_hedge"]["token"]
data["accountID"] = config["oanda_demo_hedge"]["account_id"]
data["api"] = API(access_token = data["access_token"])
r = accounts.AccountDetails(data["accountID"])

data["api"].request(r)
data["account_data"] = r.response
data["account_balance"] = float(data["account_data"]["account"]["balance"])

data["invest_amount"] = data["account_balance"] / data["balance_invest_ratio"]
data["order_amount"] = int(np.floor(data["invest_amount"] * data["leverage"]))


print(f'account_balance: {data["account_balance"]} ')
print(f'invest_amount: {data["invest_amount"]} ')
print(f'order_amount: {data["order_amount"]} ')